In [9]:
# Importing the important Libraries
import pandas as pd
import math
from surprise import Reader, Dataset, evaluate
import os
import random
from surprise import SlopeOne
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [10]:
#Setting the path for file
os.chdir("C:/Users/Swati Arora/Data Science using Python/Movie Recommender System- Pybots/Data")
file_path = os.path.expanduser("C:/Users/Swati Arora/Data Science using Python/Movie Recommender System- Pybots/Data")

In [11]:
df = pd.read_csv('ratings.dat',sep='::', encoding='latin-1',names=['UserID','MovieID','Rating','Timestamp'])
df.columns

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


Index(['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype='object')

In [12]:
#shuffling the dataset to randomly select the userid for test and train data
df = shuffle(df)

In [13]:
#Dividing the data ino test and train
train, test = train_test_split(df, test_size=0.1)

In [14]:
len(train)/len(df)

0.8999999000208956

In [15]:
#Writing the test and train data into a CSV file.
test.to_csv("Test_dataset.csv",sep=',')
train.to_csv("Train_dataset.csv",sep=',')

In [16]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))
# Coverting the pandas data frame to slop one specific alogrith format.
data = Dataset.load_from_df(train[['UserID','MovieID','Rating']], reader)

In [17]:
#Initializing the slopeone alogorithm for model building
algo=SlopeOne()

In [18]:
# Retrieve the trainset.
trainset = data.build_full_trainset()

In [19]:
#train the Model
algo.train(trainset)

In [20]:
#Testing the model on train set
x=test
x.index = pd.RangeIndex(len(x.index))
x=x.drop('Timestamp',axis=1)

In [21]:
#Using the trained model to predict results for test Data set
x['Pred_rating']=1
for i in range(0,len(test)):
    userid=x.iloc[i]['UserID']
    itemid=x.iloc[i]['MovieID']
    actual_rating=x.iloc[i]['Rating']
    x.iloc[[i],[3]]=algo.predict(userid, itemid, actual_rating).est

In [22]:
x.to_csv("Predicted_rating.csv",sep=',')

In [23]:
#Calculating RMSE & MAE
RMSE=0
MAE=0
for i in range(0,len(x)):
    RMSE=RMSE+math.pow((x.iloc[i]['Pred_rating']-x.iloc[i]['Rating']),2)
    MAE=MAE+abs(x.iloc[i]['Pred_rating']-x.iloc[i]['Rating'])

In [24]:
RMSE=math.sqrt(RMSE)/len(x)
MAE=MAE/len(x)
print("RMSE:", RMSE)
print("MAE:",MAE)

RMSE: 0.0028630170261709676
MAE: 0.713873523791


In [25]:
#Applying K-Fold to find best model
# Split data into 5 folds
data.split(n_folds=3)
algo = SlopeOne()
evaluate(algo, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SlopeOne.

------------
Fold 1
RMSE: 0.9079
MAE:  0.7153
------------
Fold 2
RMSE: 0.9096
MAE:  0.7165
------------
Fold 3
RMSE: 0.9095
MAE:  0.7167
------------
------------
Mean RMSE: 0.9090
Mean MAE : 0.7162
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.71527449137032351,
                             0.71648387400934732,
                             0.71674540997237546],
                            'rmse': [0.90785426887076948,
                             0.90962945384021454,
                             0.9094537748391921]})

In [26]:
# Recommending Top Five Movies to a random user:
i=random.randrange(0, len(df), 1)
i=df.iloc[i]['UserID']
user=df[(df.UserID)==i]

In [27]:
movies=pd.read_csv('movies.dat',sep='::',encoding='ANSI',names=['MovieID','Title','Genres'])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [28]:
#Merging user rating data with movie data to identify which movies user has not wtached
user_movies=pd.merge(movies,user,left_on='MovieID',right_on='MovieID',how='left')

In [29]:
user_movies=user_movies[pd.isnull(user_movies['Rating'])]
user_movies['UserID']=i
user_movies=user_movies.drop('Timestamp',axis=1)
user_movies=user_movies.drop('Genres',axis=1)
user_movies.index = pd.RangeIndex(len(user_movies.index))

In [30]:
#Recommending top five movies
y=user_movies
for i in range(0,len(y)):
    userid=y.iloc[i]['UserID']
    itemid=y.iloc[i]['MovieID']
    actual_rating=y.iloc[i]['Rating']
    y.iloc[[i],[3]]=algo.predict(userid, itemid, actual_rating).est

In [31]:
# Sorting the movies for a particular user on the basis of their rating.
y=y.sort_values(['Rating'],ascending=[0])
print("Top 5 Recommended movies For User:",y.iloc[0]['UserID'])
y.head(5)

Top 5 Recommended movies For User: 601


,MovieID,Title,UserID,Rating
502,572,Foreign Student (1994),601,5.0
2971,3382,Song of Freedom (1936),601,5.0
2913,3323,Chain of Fools (2000),601,5.0
577,657,Yankee Zulu (1994),601,5.0
508,578,"Hour of the Pig, The (1993)",601,5.0
